# Computational Theory Assement Problems

## Problem 1: Binary Words and Operations

In [ ]:
import numpy as np
#1.
# ^ = bitwise XOR
def Parity(x,y,z):
    """
    Performs bitwise XOR(exclusive or) operation on each bit position of the 3 values
    x == 0 y == 0 z == 0 result == 0
    x == 1 y == 0 z == 0 result == 1 (returns 1 when odd number of 1's)
    x == 1 y == 1 z == 0 result == 0 (returns 0 when even number of 1's)
    x == 1 y == 1 z == 1 result == 1
    Arg:

    Parameters: 
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned  integer

    Returns:
    np.uint32
        value made up of results for each bit position of the 3 input XOR
    """
    return np.uint32(x) ^ np.uint32(y) ^ np.uint32(z)
#2.
# & = bitwise and 
# ~ flips all bits i.e 0x00000000 -> 0xFFFFFFFF
def Ch(x,y,z):
    """
    returns new value built from the taken bits of y and z based on the state of x

    For each bit position in x check if the bit is 1.
    if it is, take the corresponding bit in y 
    if not, take the corresponding bit in z

    Parameters
    ----------
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned integer

    Returns
    ---------
    np.unit32 
        value made from z and y based on the state of x
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (~x & z)
#3.
def Maj(x,y,z):
    """ 
    Returns 32 bit integer bitwise majority

    For each bit position, returns 1 if two or more of the three inputs 
    have that bit set to 1 else returns 0.

    Parameters
    ---------
    X: int 
    First 32 bit unsigned integer 
    Y: int
    Second 32 bit unsigned integer
    Z: int 
    Third 32 bit unsigned integer

    Returns
    ------- 
    np.uint32
        value made up of the majority of each bit position of the 3 values
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    #For each bit position perform a bitwise AND between each of the 3 values
    #which returns 1 if both bits are 1 or 0 in all other cases, then perform a 3 input XOR
    #using the values returned from the AND comparisons 

    return (x & y) ^ (x & z) ^ (y & z)
#4.
# | = bitwise inclusive or
# W = number of bits 
# >> = shifts each bit to the right bits on the right fall off and are replaced on the left by 0's
# i.e n = 3   11000-->00011
# << = shifts each bit to the left bits on the left fall off and are replaced on the right by 0's
#i.e n = 3 11001->01000
def ROTR(x,n):
    """ 
    Executes  rotate right (circular right shift) operation on 32 bit unsigned integer
    Rotates all bits in X to the right by n.Any bits that fall off are wrapped to the left
    end

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 
    N: int 
    number of bits to rotate x by 0-31

    Returns
    ------- 
    np.uint32
        Resulting rotation of x by n bit positions
        
    """
    x = np.uint32(x)
    n = np.uint32(n)
    w = 32
    #shifts each bit in x to the right by n positions
    #left shift the bits of x by the bit size of x-n(32-n) 
    #combine both shifted values using bitwise OR to return the rotated result
    #& 0xFFFFFFFF Ensures results remain within 32bits
    return ((x >> n) | (x << (w - n))) & 0xFFFFFFFF
def SHR(x,n):
    """Executes the right shift operation on a 32-bit integer.
    shifts each bit position of x by n positions.Any bit that falls off is replaced by a 0 from the left end
    
    Parameters
    ---------
    X: int 
    32 bit unsigned integer 
    N: int 
    number of bits to shift x by 0-31

    Returns
    ------- 
    np.uint32
        Resulting shift of x by n bit positions
    
    """
    x = np.uint32(x)
    n = np.uint32(n)
    #shift bit positions of x by n positions and return the shifted 32 bit integer
    return np.uint32( x >> n)
def Sigma0(x):
    """
    SHA-256 Sigma0 (Σ₀)
    Performs 3 input XOR using 3 seperate right rotated values of x,
      each with differing rotation sizes(2, 13, and 22 bit positions)

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 

    Returns
    -------
    np.uint32
        Result of the 3 input XOR on the rotated values
    """
    x = np.uint32(x)
    return ROTR(x,2) ^ ROTR(x,13) ^ ROTR(x,22)
#5.
def Sigma1(x):
    """
    SHA-256 Sigma1 (Σ₁) function

    Performs 3 input XOR using 3 seperate right rotated values of x,
      each with differing rotation sizes(6, 11, and 25 bit positions) 
      

    Parameters
    ---------
    X: int 
    32 bit unsigned integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 3 rotated values 
    """
    x = np.uint32(x)
    return ROTR(x,6) ^ ROTR(x,11) ^ ROTR(x,25)
#6.
def Sigma0_2(x):
    """
    SHA-256 sigma0 (σ₀)
    Performs 3 input XOR using 2 seperate right rotated values of x,
      each with differing rotation sizes(7, and 18 bit positions) 
      and one right shifted value of x shifted by 3 bit positions

    Parameters
    ---------
    X: int 
    32 bit integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 2 rotated values of x and 1 right shifted value of x
    """
    x = np.uint32(x)
    return ROTR(x,7) ^ ROTR(x,18) ^ SHR(x,3)
#7.
def Sigma1_2(x):
    """
    SHA-256 sigma1 (σ₁)
    Performs 3 input XOR using 2 seperate right rotated values of x
      each with differing rotation sizes(17, and 19 bit positions) 
      and one right shifted value of x shifted by 10 bit positions
    Parameters
    ---------
    X: int 
    32 bit integer 

    Returns
    --------
    np.uint32
        Result of the 3 input XOR on the 2 rotated values of x and 1 right shifted value of x
    """
    x = np.uint32(x)
    return ROTR(x,17) ^ ROTR(x,19) ^ SHR(x,10)

print("Right Rotation of 16 by 2 is ")    
print(ROTR(16,2))
print("Left Rotation of 16 by 2 is ")   
print(ROTL(16,2))

#Testing and explanations needed 

Right Rotation of 16 by 2 is 
4
Left Rotation of 16 by 2 is 
64


## Problem 2: Fractional Parts of Cube Roots

In [ ]:

def primes(n):
    x = 0
    y = 2
    flag = False
    for j in range(2, np.floor(np.sqrt(y)) + 1):
        if(y % j == 0):
            flag = False
            break
    if(flag):
        #Create an array to add the primes then return at the end
        x+=1
    i+=1        

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords